# References

##### [Seasonality lesson from Kaggle's Time Series Course](http://www.kaggle.com/ryanholbrook/seasonality)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-mar-2022/train.csv")

In [ ]:
test = pd.read_csv("../input/tabular-playground-series-mar-2022/test.csv")
sample_submission = pd.read_csv("../input/tabular-playground-series-mar-2022/sample_submission.csv")

In [ ]:
train

In [ ]:
train["date"]=pd.to_datetime(train["time"])
train["date2"]=pd.to_datetime(train["time"])

# EDA

In [ ]:
print("train has %s column and %s row and column names: %s"%(train.shape[1],train.shape[0],train.columns.tolist()))
print("test has %s column and %s row and column names: %s"%(test.shape[1],test.shape[0],test.columns.tolist()))
print("sample_submission has %s column and %s row and column names: %s"%(sample_submission.shape[1],sample_submission.shape[0],sample_submission.columns.tolist()))

In [ ]:
train.head()

In [ ]:
train

In [ ]:
test

In [ ]:
train.info()

In [ ]:
train.drop("row_id",axis = 1).describe()

In [ ]:
print("NaN values: %s "%train.isnull().sum().sum())

In [ ]:
train.groupby("direction").count()

In [ ]:
train.groupby("direction").nunique()

In [ ]:
sns.scatterplot(data=train,x="x",y="y")

In [ ]:
directions=train.groupby("direction").nunique().index.tolist()
xs=train.groupby("x").nunique().index.tolist()
ys=train.groupby("y").nunique().index.tolist()

In [ ]:
point_df_0_0_EB = train[(train["x"]==0) & (train["y"]==0)&(train["direction"]=="EB")]
point_df_0_0_EB

In [ ]:
point_df_0_0_EB.isnull().sum().sum()

In [ ]:
point_df_0_0_EB.index

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
point_df_0_0_EB["MA3"] = point_df_0_0_EB.congestion.rolling(window=3).mean()
point_df_0_0_EB["MA3_shift"] = point_df_0_0_EB["MA3"].shift(3)

In [ ]:
plt.figure(figsize=(22,6))
sns.lineplot(data=point_df_0_0_EB[100:500],x="date",y="congestion")
sns.lineplot(data=point_df_0_0_EB[100:500],x="date",y="MA3")
sns.lineplot(data=point_df_0_0_EB[100:500],x="date",y="MA3_shift")


In [ ]:
list_df_dict = {}
for direct in directions:
    for x in xs:
        for y in ys:
            df_name = "point_df_"+str(x)+"_"+str(y)+"_"+direct #point_df_0_0_EB >> x=0 , y=0 , direction="EB"
            locals()[df_name] = train[(train["x"]==x) & (train["y"]==y)&(train["direction"]==direct)]
            locals()[df_name]["MA7"] = locals()[df_name]["congestion"].ewm(com=10.0,min_periods=10).mean()
            locals()[df_name]["MA3"] = locals()[df_name].congestion.ewm(com=2.0,min_periods=2).mean()
            locals()[df_name]["MA36"] = locals()[df_name].congestion.rolling(window=36).mean()
            locals()[df_name]["noise with roll MA5"] = (locals()[df_name].congestion - locals()[df_name].MA7).ewm(com=36.0,min_periods=36).mean()
            locals()[df_name]["noise+MA7"] =locals()[df_name]["MA7"]+locals()[df_name]["noise with roll MA5"]
            locals()[df_name]["noise+MA7 shift"] = locals()[df_name]["noise+MA7"].shift(3)          
            locals()[df_name]=locals()[df_name].dropna()
            locals()[df_name]=locals()[df_name].set_index("date").to_period("min")
            if locals()[df_name].notna().sum().sum()>locals()[df_name].isnull().sum().sum():
                list_df_dict[df_name] = locals()[df_name]
#                 plt.figure(figsize=(20,3))
#                 sns.lineplot(data=locals()[df_name][2000:2750],x="date2",y="congestion",label="congestion")
#                 sns.lineplot(data=locals()[df_name][2000:2750],x="date2",y="MA36",label="ma36")
#                 sns.lineplot(data=locals()[df_name][2000:2750],x="date2",y="MA7",label="ma7")
#                 sns.lineplot(data=locals()[df_name][2000:2750],x="date2",y="noise+MA36",label="noise+ma3")
#                 sns.lineplot(data=locals()[df_name][2000:2750],x="date2",y="noise with roll MA5",label="noise with roll MA5")

#                 plt.title(df_name)
#                 plt.legend()
#                 plt.show()
            else:
                pass
print("total point_df count: %s"%len(list(list_df_dict.values())))      

In [ ]:
def fourier_features(index, freq, order):
    time = np.arange(len(index), dtype=np.float32)
    k = 2 * np.pi * (1 / freq) * time
    features = {}
    for i in range(1, order + 1):
        features.update({
            f"sin_{freq}_{i}": np.sin(i * k),
            f"cos_{freq}_{i}": np.cos(i * k),
        })
    return pd.DataFrame(features, index=index)


# Compute Fourier features to the 4th order (8 new features) for a
# series y with daily observations and annual seasonality:
#
# fourier_features(y, freq=365.25, order=4)

In [ ]:
from warnings import simplefilter

simplefilter("ignore")

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 5))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)
%config InlineBackend.figure_format = 'retina'


# annotations: https://stackoverflow.com/a/49238256/5769929
def seasonal_plot(X, y, period, freq, ax=None):
    if ax is None:
        _, ax = plt.subplots()
    palette = sns.color_palette("husl", n_colors=X[period].nunique(),)
    ax = sns.lineplot(
        x=freq,
        y=y,
        hue=period,
        data=X,
        ci=False,
        ax=ax,
        palette=palette,
        legend=False,
    )
    ax.set_title(f"Seasonal Plot ({period}/{freq})")
    for line, name in zip(ax.lines, X[period].unique()):
        y_ = line.get_ydata()[-1]
        ax.annotate(
            name,
            xy=(1, y_),
            xytext=(6, 0),
            color=line.get_color(),
            xycoords=ax.get_yaxis_transform(),
            textcoords="offset points",
            size=14,
            va="center",
        )
    return ax


def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta("1Y") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(
        [
            "Annual (1)",
            "Semiannual (2)",
            "Quarterly (4)",
            "Bimonthly (6)",
            "Monthly (12)",
            "Biweekly (26)",
            "Weekly (52)",
            "Semiweekly (104)",
        ],
        rotation=30,
    )
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax


In [ ]:
train_new = pd.concat(list(list_df_dict.values())).sort_values("row_id")

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
train_new.columns

In [ ]:
# 

In [ ]:
X = pd.concat([train_new[['MA7']],pd.get_dummies(train_new["direction"]),pd.get_dummies(train_new["x"],prefix="x"),pd.get_dummies(train_new["y"],prefix="y")],axis=1)
y = train_new["MA3"]

In [ ]:
def time_features(df):
    df["min"] = df.index.minute 
    df["day"] = df.index.day  
    df["hour"] = df.index.hour
    df["weekday"] = df.index.weekday
    df["h0-6"] = (df["hour"]<6).astype(int)
    df["h6-12"] = ((df["hour"]<12)&(df["hour"]>6)).astype(int)
    df["h12-18"] = ((df["hour"]<18)&(df["hour"]>12)).astype(int)
    df["h18-00"] = ((df["hour"]<=23)&(df["hour"]>18)).astype(int)
    return df

In [ ]:
X =time_features(X)

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
    
input_shape = [X.shape[1],]

In [ ]:
keras.backend.clear_session()

In [ ]:
normalizer = keras.layers.Normalization(input_shape=input_shape, axis=None)
normalizer.adapt(X)

In [ ]:
def model(norm):
    model = keras.Sequential([
        norm,
        keras.layers.Dense(64,activation="relu"),
        keras.layers.Dense(64,activation="relu"),
        keras.layers.Dense(1)
      ])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),loss="mae",metrics=["mae","mse"])
    model.summary()
    return model

In [ ]:
forecast_model = model(normalizer)

In [ ]:
def scheduler(epoch, lr):
      if epoch < 3:
        return lr
      else:
        return lr * tf.math.exp(-0.01)

In [ ]:
batch_size=512
epochs=10

In [ ]:
lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
history = forecast_model.fit(X.values, y.values, batch_size=batch_size, epochs=epochs, callbacks=[lr],validation_split=0.1,shuffle=True)

In [ ]:
_,ax = plt.subplots( ncols=2,nrows=1,figsize=(16,4))
ax[0].set_xlabel("epochs")
ax[0].set_ylabel("loss")
ax[0].set_title("final val_loss %1.4f"%(forecast_model.history.history["val_loss"][-1:][0]))
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("accuracy")
ax[1].set_title("final val_mae %1.4f"%(forecast_model.history.history["mae"][-1:][0]))
pd.DataFrame([forecast_model.history.history["mae"],forecast_model.history.history["val_mae"]],index=["mae","val_mae"]).T.plot(ax=ax[1])
pd.DataFrame([forecast_model.history.history["loss"],forecast_model.history.history["val_loss"]],index=["loss","val_loss"]).T.plot(ax=ax[0])


In [ ]:
from xgboost import XGBRegressor
from sklearn.tree import ExtraTreeRegressor

In [ ]:
forecast_model2 = ExtraTreeRegressor()
forecast_model3 = XGBRegressor()

In [ ]:
forecast_model2.fit(X,y)
forecast_model3.fit(X,y)

In [ ]:
train_columns = ['congestion',"noise+MA7","MA7","MA36","noise+MA7 shift"]

In [ ]:
list_point_df = list(list_df_dict.values())
list_point_names = list(list_df_dict.keys())

# model training

In [ ]:
from IPython.display import clear_output
import time

from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from catboost import CatBoostRegressor

In [ ]:
regression_model=ExtraTreeRegressor()

In [ ]:
from statsmodels.tsa.deterministic import CalendarTimeTrend,CalendarSeasonality

In [ ]:
def calendar(inds,calPltCount):
    fores2 = []
    preds2 = []
    for ind in inds:

        preds = []
        forecasts = []
        Maes=[]


        for df_loc,df in enumerate(list_point_df):#
#             print("Calculating for "+list_point_names[df_loc])
            fore_start = 12500
            fore_end = 13000
            y_fore_end = fore_end-fore_start-1 
            Xlast_fore_end = list_point_df[0][train_columns].shape[0]-fore_end
            forward_steps = Xlast_fore_end*65 + test.shape[0]*5
            X = list_point_df[df_loc][train_columns][0:fore_start]

            yy = list_point_df[df_loc][ind][0:fore_start]
            yy2 = list_point_df[df_loc]["congestion"][0:fore_start]
            y_true_forecast = list_point_df[df_loc]["congestion"][fore_start+1:fore_end]

            XX = list_point_df[df_loc][train_columns]

            X["day"] = X.index.dayofweek  # the x-axis (freq)
            X["week"] = X.index.week  # the seasonal period (period)
            X["hour"] = X.index.hour

            dp = DeterministicProcess(
                                        index=X.index,
                                        constant=False,               # dummy feature for bias (y-intercept)
                                        order=1,                     # trend (order 1 means linear)
        #                                 seasonal=True,               # weekly seasonality (indicators)
                                        additional_terms=[fourier,cal_seas_gen,cal_trend_gen], # annual seasonality (fourier)
                                        drop=False                 # drop terms to avoid collinearity

                                        )

#             print("2. First Plot")
        #     fig, (ax0, ax1,ax2) = plt.subplots(3, 1, figsize=(24, 8))
        #     seasonal_plot(X, y="congestion", period="week", freq="hour", ax=ax1);
        #     seasonal_plot(X, y="congestion", period="week", freq="day", ax=ax2);

        #     plt.show()
#             print("3. Creting DP")
            X = dp.in_sample()  


#             print("X_train shape: ",X.shape,"y_train shape: ",yy.shape )
            model = regression_model
            model2 = regression_model
            model3 = regression_model
#             print("6. ",list_point_names[df_loc]+"'s fitting...")

            _= model.fit(X, yy)
#             print("7. Forecasting")
            y_pred = pd.Series(model.predict(X), index=yy.index)

            X_fore = dp.out_of_sample(steps=(forward_steps)) 
            y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)



            #     plt.figure(figsize=(8,3))
        #     sns.regplot(x=y_pred,y=yy,line_kws={"color":"black"})
        #     plt.title("X, y regression plot")
        #     plt.show()
        #     plt.figure(figsize=(8,3))
        #     sns.histplot((y_pred-yy),kde=True,bins=180)
        #     plt.title("X, y error histogram")
        #     plt.show()

#             print(list_point_names[df_loc]+'predict_MAE:', mean_absolute_error(y_pred,yy))
#             print(list_point_names[df_loc]+'predict_MSE:', mean_squared_error(y_pred,yy))

#             print("8. Second Plot")
            if df_loc<calPltCount:
                plt.figure(figsize=(22,5),dpi=120)

                ax = yy.plot(color="black", style='.', title=list_point_names[df_loc]+" congestion - Hourly Forecast")
                ax = y_pred.plot(ax=ax, label=list_point_names[df_loc] +" Hourly Predict")
                ax = y_true_forecast.plot(color="yellow",style='.',ax=ax, label=list_point_names[df_loc] +" True Forecast")
                ax = y_fore.plot(ax=ax, label=list_point_names[df_loc] +" Hourly Forecast", color='C3')

                _ = ax.legend()
                plt.show()
            else:
                pass
#             print("y_fore[0:%s] shape:"%y_fore_end , y_fore[0:y_fore_end].shape ,"y_true_forecast shape:",y_true_forecast.shape)
#             print(list_point_names[df_loc]+' forecast_MAE:', mean_absolute_error(y_fore[0:y_fore_end],y_true_forecast))
#             print(list_point_names[df_loc]+' forecast_MSE:', mean_squared_error(y_fore[0:y_fore_end],y_true_forecast))
            mae=mean_absolute_error(y_fore[0:y_fore_end],y_true_forecast)
            df_pred = pd.DataFrame(y_pred,columns=[ind])
            df_pred["x"] = int(list_point_names[df_loc][9])
            df_pred["y"] = int(list_point_names[df_loc][11])
            df_pred["direction"] = list_point_names[df_loc][13:]

            df_fore = pd.DataFrame(y_fore,columns=[ind])
            df_fore["x"] = int(list_point_names[df_loc][9])
            df_fore["y"] = int(list_point_names[df_loc][11])
            df_fore["direction"] = list_point_names[df_loc][13:]

            preds.append(df_pred)
            forecasts.append(df_fore)
#             print("9."+list_point_names[df_loc]+" finished")
            Maes.append(mae)
#             print("")
#             print("df: %s Average Forecast MAE : %1.5f"%(64-df_loc,sum(Maes)/len(Maes)))
#             print("")
#         print("All Done.")
#         print("Average Forecast MAE : %1.5f"%(sum(Maes)/len(Maes)))
        df_prediction = pd.concat(preds)
        df_prediction= df_prediction.to_timestamp()
        df_forecast = pd.concat(forecasts)
        df_forecast= df_forecast.to_timestamp()
        fores2.append(df_forecast)
        preds2.append(df_prediction)
        return fores2, preds2

In [ ]:
fourier = CalendarFourier(freq="W", order=10)
cal_seas_gen = CalendarSeasonality("H", "W")
cal_trend_gen = CalendarTimeTrend("D", True, order=0)
inds=['MA7']

In [ ]:
# fores2,preds2 = calendar(inds=inds,calPltCount=3)

In [ ]:
def forecast(fores2,preds2,train):
    df_prediction = pd.concat(preds2,axis=1)
    df_forecast = pd.concat(fores2,axis=1)

    x2 = df_forecast.iloc[:,1]
    y2 = df_forecast.iloc[:,2]
    direction = df_forecast.iloc[:,3]
    df_forecast = df_forecast.drop(["x","y","direction"],axis=1)
    df_forecast["x"] = x2
    df_forecast["y"] = y2
    df_forecast["direction"] = direction

    df_forecast_mae = df_forecast.reset_index()
    df_forecast_mae = df_forecast_mae[(df_forecast_mae["index"].astype('int64')/1000000000)%(60*20)==0]
    df_forecast_mae = df_forecast_mae.sort_values(["index","x","y","direction"])

    X1= pd.concat([df_forecast_mae[['MA7',"index"]],pd.get_dummies(df_forecast_mae["direction"]),pd.get_dummies(df_forecast_mae["x"],prefix="x"),pd.get_dummies(df_forecast_mae["y"],prefix="y")],axis=1)
    X1.set_index("index",inplace=True)

    X2 =time_features(X1)

    pred = forecast_model.predict(X2).reshape(1,-1)[0]
    pred2 = forecast_model2.predict(X2)
    pred3 = forecast_model3.predict(X2)

    df_forecast_mae["congestion"]=pred
    df_forecast_mae["congestion2"]=pred2
    df_forecast_mae["congestion3"]=pred3

    t1 =df_forecast_mae["index"].iloc[0].strftime("%Y-%m-%d, %H:%M:%S")
    t2 =train["date"].iloc[-1].strftime("%Y-%m-%d, %H:%M:%S")

    slice1 = train.set_index("date").loc[t1:t2]["congestion"].shape[0]

    foreCongestion = df_forecast_mae.set_index("index").loc[t1:t2][:slice1]["congestion"]
    foreCongestion2 = df_forecast_mae.set_index("index").loc[t1:t2][:slice1]["congestion2"]
    foreCongestion3 = df_forecast_mae.set_index("index").loc[t1:t2][:slice1]["congestion3"]

    trueCongestion = train.set_index("date").loc[t1:t2]["congestion"]
    tfMae = mean_absolute_error(trueCongestion,foreCongestion)
    extMae = mean_absolute_error(trueCongestion,foreCongestion2)
    xgbMae = mean_absolute_error(trueCongestion,foreCongestion3)
    
    print(f" tensorflow mae: {tfMae:.3f},extra trees mae: {extMae:3f}, xgb mae: {xgbMae:3f}" )
    return df_forecast_mae, tfMae, extMae, xgbMae

In [ ]:
# forecast(fores2=fores2, preds2=preds2, train=train)

In [ ]:
CalSeasFr = ["H","H","D","M","M"]
CalSeasPer = ["D","W","W","Q","A"]
CalFou = ["D","W","M","B","Q","A"]
CalTT = ["D","W","M","B","Q","A"]

In [ ]:
df_lfore = []
tfm = []
ext = []
xgb = []
params = []
num = 180
for fou in CalFou:
    for tt in CalTT:
        for freq,period in zip(CalSeasFr,CalSeasPer):
            t1 = time.time()
            fourier = CalendarFourier(freq=fou, order=10)
            cal_seas_gen = CalendarSeasonality(freq, period)
            cal_trend_gen = CalendarTimeTrend(tt, True, order=0)
            fores2,preds2 = calendar(inds=inds,calPltCount=0)
            print(f" for CalSeasFr: {freq}, CalSeasPer: {period}, CalFou: {fou}, CalTT: {tt},")
            df_forecast_mae, tfMae, extMae, xgbMae = forecast(fores2=fores2, preds2=preds2, train=train)
            num-=1
            
            print(f" remaining calc: {num}, remaining time:  ~ {int((time.time()-t1)*(num)/60)} min {int((((time.time()-t1)*(num)%60)%1)*60)} sec. ")
            param = {"CalFou": fou ,  "CalTT" : tt , "CalSeasFr": freq , "CalSeasPer" : period}
            params.append(param)
            df_lfore.append(df_forecast_mae)
            tfm.append(tfMae)
            ext.append(extMae)
            xgb.append(xgbMae)

In [ ]:

print(f"best results | tensorflow mae: {tfm[np.argmin(tfm)]:.3f} argmin:{np.argmin(tfm)}  , extra trees mae: {ext[np.argmin(ext)]:3f} argmin:{np.argmin(ext)}  , xgb mae: {xgb[np.argmin(xgb)]:3f} argmin:{np.argmin(xgb)} " )

In [ ]:
params[np.argmin(tfm)]

In [ ]:
df_forecast_mae = df_lfore[np.argmin(tfm)]

In [ ]:
df_forecast_mae["congestion4"] = pd.concat(df_lfore,axis=1).set_index(pd.concat(df_lfore,axis=1).iloc[:,0])["congestion"].mean(axis=1).values

In [ ]:
df_forecast_mae = df_forecast_mae.set_index("index").loc["1991-09-30 12:00:00":"1991-09-30 23:40:00"]

In [ ]:
df_forecast_mae=df_forecast_mae.sort_values(["index","x","y","direction"])

In [ ]:
test=test.sort_values(["time","x","y","direction"])

In [ ]:
df_forecast_mae.head()

In [ ]:
test["congestion"]=df_forecast_mae["congestion4"].values.astype(int)

In [ ]:
test=test.sort_values("row_id")

In [ ]:
test["congestion"].mean()

In [ ]:
train["congestion"].mean()

In [ ]:
sample_submission["congestion"] = test["congestion"].values

In [ ]:
sample_submission.to_csv("submission.csv",index=False)

In [ ]:
!head submission.csv

# References

##### [Seasonality lesson from Kaggle's Time Series Course](http://www.kaggle.com/ryanholbrook/seasonality)